# V4: True two layers(two pins on the different layer)

In [1]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
filepath_out='../output/bench5.router'
netlist_file_path = '../benchmark/bench5.nl'
gridfile_path  = '../benchmark/bench5.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 10 
 via_penalty: 20
size of layer1: (127, 317)
size of layer2: (127, 317)


In [2]:
# plot problem 根据grid文件可视化障碍物和每个格子的cost，同时标出source和target，可以显示两层
plot_problem("../output/bench4_problem.jpg",columns,rows,layer1_grid_original,layer2_grid_original,nets)

In [2]:
def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    #print(path)
    if path:               # 判断是否为空 如果path为None 后面不可迭代会报错 5.22
        for cell in path:
            x, y, _ = cell
            layer1_grid[x][y] = -1

In [3]:
def get_cell_cost(layer_grid, cell,path_tmp,bend_penalty,via_penalty):  
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer_grid[x][y] != 1:
        cell_cost = layer_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer:
            cell_cost += via_penalty
        elif (prev_x != x or prev_y != y):  #v4，层变化的，加穿孔cost(woods5/22) 已加(justin5/22)
            cell_cost += bend_penalty
    return cell_cost

# modified by junjun
def expand_source_to_target(rows, columns, layer1_grid,layer2_grid, source, target,bend_penalty,via_penalty):#5.22 把layer2——grid输入，计算cost加入条件判断(woods5/22)
    wavefront = {}
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    wavefront[source_tuple] = 0
    costs[source_tuple] = 0  # Initial cost for the source cell is 0

    while wavefront:
        # get lowest cost cell on a wavefront structure
        current_cell = sorted(wavefront.items(),key=lambda s:int(s[1]))[0][0]

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])

            if neighbor_tuple not in visited:
                path_tmp= reconstruct_path(source_tuple, current_cell, parents)
                # Calculate the cost to reach the neighbor cell
                if(neighbor_tuple[2]== 1):
                  cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                if(neighbor_tuple[2]== 2):
                  cost = costs[current_cell] + get_cell_cost(layer2_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                # ignore blocks
                if cost!= np.inf:
                    if neighbor_tuple not in wavefront.keys() or costs[neighbor_tuple] > cost:
                        costs[neighbor_tuple] = cost
                        parents[neighbor_tuple] = current_cell

                    if neighbor_tuple not in wavefront.keys():
                        # add cell N to waveform, indexed by pathcost
                        wavefront[neighbor_tuple]=cost         

        visited.add(current_cell)    
        del wavefront[current_cell]                  
    return None,None

def get_neighbors(rows, columns, cell):
    x, y, layer = cell
    neighbors = []
    if y < rows - 1:
        neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
    if x < columns - 1:
        neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if x > 0:
        neighbors.append({'x': x - 1, 'y': y, 'layer': layer})
    neighbors.append({'x': x, 'y': y, 'layer': 3-layer}) ##v4加两个方向如果为layer==1 ，up，==2，down(woods5/22)  已加(justin5/22)                                                           
    return neighbors

In [4]:
def true_two_layer_router(rows, columns, layer1_grid,layer2_grid, nets,bend_penalty,via_penalty):#修改了函数名，把layer2——grid输入(woods5/22)
    routing_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = -1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = -1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = -1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = -1
            #这里区分不同层的pin，if语句有点多(woods5/22)
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
    
    for net in nets:#在V4版本中不能分开处理
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = 1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = 1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = 1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = 1
        print('Routing net:',net_id)
        path,costs = expand_source_to_target(rows, columns, layer1_grid,layer2_grid, pin1, pin2,bend_penalty,via_penalty) 
        costs = {}
        if path is not None:
            mark_path_on_grid(layer1_grid,path)
            routing_table[net_id] = path
    #plot_path('../output/bench4_v3_layer1.jpg',columns=columns,rows=rows,block_list=np.argwhere(layer1_grid==-1),path_dict=routing_table) 
    #此处要重写plot函数，实现一个函数，根据path标记和绘制grid1/2的图(woods5/22)
        
    return routing_table,costs

In [5]:
layer1_grid = layer1_grid_original
layer2_grid = layer2_grid_original
routing_table,costs=true_two_layer_router(rows, columns, layer1_grid.T,layer2_grid.T, nets,bend_penalty,via_penalty)
#generate_output_file(filepath_out,net_num,routing_table)
routing_table

Routing net: 1
Routing net: 2
Routing net: 3
Routing net: 4
Routing net: 5
Routing net: 6
Routing net: 7
Routing net: 8
Routing net: 9
Routing net: 10
Routing net: 11
Routing net: 12
Routing net: 13
Routing net: 14
Routing net: 15
Routing net: 16
Routing net: 17
Routing net: 18
Routing net: 19
Routing net: 20
Routing net: 21
Routing net: 22
Routing net: 23
Routing net: 24
Routing net: 25
Routing net: 26
Routing net: 27
Routing net: 28
Routing net: 29
Routing net: 30
Routing net: 31
Routing net: 32
Routing net: 33
Routing net: 34
Routing net: 35
Routing net: 36
Routing net: 37
Routing net: 38
Routing net: 39
Routing net: 40
Routing net: 41
Routing net: 42
Routing net: 43
Routing net: 44
Routing net: 45
Routing net: 46
Routing net: 47
Routing net: 48
Routing net: 49
Routing net: 50
Routing net: 51
Routing net: 52
Routing net: 53
Routing net: 54
Routing net: 55
Routing net: 56
Routing net: 57
Routing net: 58
Routing net: 59
Routing net: 60
Routing net: 61
Routing net: 62
Routing net: 63
R

{1: [(56, 49, 1),
  (57, 49, 1),
  (57, 48, 1),
  (58, 48, 1),
  (58, 47, 1),
  (58, 46, 1),
  (58, 45, 1),
  (58, 44, 1),
  (58, 43, 1),
  (58, 42, 1),
  (58, 41, 1),
  (58, 40, 1),
  (58, 39, 1),
  (58, 38, 1),
  (58, 37, 1),
  (58, 36, 1),
  (58, 35, 1),
  (58, 34, 1),
  (58, 33, 1),
  (58, 32, 1),
  (58, 31, 1)],
 2: [(58, 49, 1),
  (59, 49, 1),
  (60, 49, 1),
  (61, 49, 1),
  (62, 49, 1),
  (62, 48, 1),
  (63, 48, 1),
  (64, 48, 1),
  (65, 48, 1),
  (66, 48, 1),
  (67, 48, 1),
  (68, 48, 1),
  (69, 48, 1),
  (70, 48, 1),
  (71, 48, 1),
  (72, 48, 1),
  (73, 48, 1),
  (74, 48, 1),
  (75, 48, 1),
  (76, 48, 1),
  (77, 48, 1),
  (78, 48, 1),
  (79, 48, 1),
  (80, 48, 1),
  (81, 48, 1),
  (81, 47, 1),
  (81, 46, 1),
  (81, 45, 1),
  (81, 44, 1),
  (81, 43, 1),
  (81, 42, 1),
  (81, 41, 1),
  (81, 40, 1),
  (81, 39, 1),
  (81, 38, 1),
  (81, 37, 1),
  (81, 36, 1),
  (81, 35, 1),
  (81, 34, 1),
  (81, 33, 1),
  (81, 32, 1),
  (81, 31, 1)],
 3: [(10, 49, 1),
  (10, 50, 1),
  (10, 51, 1),

In [6]:
cnt = 0
for item in routing_table.keys():
    if routing_table[item]:
        cnt += 1
cnt

128